# News similarity modelling

[Dataset](https://www.kaggle.com/c/newssimil)

In [1]:
import pandas as pd
import csv
import numpy as np
import nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrgolubev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/andrgolubev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
train_file = 'news_similarity_train.csv'

In [9]:
news_df = pd.read_csv(train_file, sep=",",
                      engine="c",
#                       names=['post_id','published_at','title','subtitle','content'],
#                       dtype={
#                           'post_id': np.int32,
#                           'published_at': np.datetime64,
#                           'title': str,
#                           'subtitle': str,
#                           'content': str
#                       }
#                       skiprows=skip
                     )

In [10]:
news_df.head()

,post_id,published_at,title,subtitle,content
0,1,2017-01-01T00:05:00Z,Полиция Стамбула собирается штурмовать ночной ...,"Согласно последним данным, в здании может нахо...",<p>Полиция Стамбула собирается штурмовать ночн...
1,2,2017-01-01T00:07:00Z,Расстрел посетителей в ночном клубе в Стамбуле...,"Ранее сообщалось, что полицейские собираются ш...","<p>Опубликовано видео, где злоумышленник расст..."
2,3,2017-01-01T00:21:00Z,Губернатор Стамбула: При нападении на ночной к...,"Ранее сообщалось, что от 40 до 60 человек полу...",<p>Губернатор Стамбула <span>Васип Шахин</span...
3,4,2017-01-01T00:29:00Z,Губернатор Стамбула назвал террористической ат...,"Согласно последним данным, погибло минимум 35 ...",<p>Губернатор Стамбула Васип Шахин назвал терр...
4,5,2017-01-01T00:47:00Z,Очевидцы: В ночной клуб в Стамбуле проходили б...,Ранее губернатор Стамбула Васип Шахин назвал п...,"<p>Посетители стамбульского клуба ""Рейна"", на ..."


In [11]:
import json
def get_json(myjson):
  try:
    return json.loads(myjson)
  except ValueError:
    return None

In [8]:
print(get_json(news_df['content'][103325]) is not None)
print(get_json(news_df['content'][0]) is not None)

True
False


In [21]:
j = get_json(news_df['content'][103297])
full_j = [row["value"]["blocks"][0]["text"] for row in j if row["type"] == "TEXT" or row["type"] == "SUBTITLE"]
"\n".join(full_j)

'21-летняя Райли Хемсон два года назад весила 115 кг\nДевушка занялась фитнесом и стала описывать всё, что с ней происходит, в соцсетях\nВо время своей работы парамедиком Хемсон сталкивалась с людьми, страдающими от избытка веса\nРайли делилась с подписчиками в "Инстаграме" не только своими успехами, но и переживаниями, ошибками\nОна начала активно пропагандировать здоровый образ жизни и идею о том, что надо любить своё тело и заботиться о нём на всех этапах жизни\nНе смотря на то что девушка является вегетарианкой, она набрала лишний вес из-за частого употребления пиццы и прочей вредной еды\nРайли вдохновила многих людей своими усилиями вести здоровый образ жизни. За год девушка потеряла 25 кг\nВ результате на страницу девушки в "Инстаграме" подписалось более 15 тыс. человек'

In [20]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(news_df['content'][16], 'html.parser')
soup.get_text().replace(u'\xa0', u' ')

'Президент США Барак Обама, генсек НАТО Йенс Столтенберг и глава европейской дипломатии Федерика Могерини выразили соболезнования в связи с терактом в Стамбуле.\n— Трагическое начало 2017 года в Стамбуле. Мои мысли сейчас с теми, кого затронуло это нападение в то время, когда они праздновали Новый год, — написал Йенс Столтенберг на своей страничке в "Твиттере".\n \nБарак Обама, в свою очередь, распорядился немедленно оказать всю необходимую помощь Турции после случившегося теракта в Стамбуле.\nФедерика Могерини также сказала, что этот год начался с трагических событий в Турции.\n— 2017 год начался с атаки на Стамбул. Наши мысли с погибшими и их родными. Мы продолжим работать над предотвращением таких трагедий, — написала Могерини в своём "Твиттере".\n \nНапомним, что в результате вооружённого нападения на элитный ночной клуб "Рейна" в Стамбуле погибли не менее 35 человек. За последними новостями по этой теме следите в нашей текстовой трансляции.'

In [85]:
news_df['content'][0]

'<p>Полиция Стамбула собирается штурмовать ночной клуб, где произошло нападение. Согласно последним данным, минимум один злоумышленник находится в здании. Ранено, по разным данным, от 40 до 60 человек.</p>\n<p>Жертвами стали не менее четырёх человек. Среди погибших — один полицейский.</p>\n<p>Во время празднеств\xa0в клубе "Рейна" находилось до 800 человек. Неизвестные произвели множество выстрелов и, как сообщается, взорвали две гранаты.</p>'

In [12]:
from pymystem3 import Mystem
m = Mystem()

In [6]:
lemmas = m.lemmatize(news_df['title'][0])

In [7]:
print(lemmas)

['полиция', ' ', 'стамбул', ' ', 'собираться', ' ', 'штурмовать', ' ', 'ночной', ' ', 'клуб', ', ', 'где', ' ', 'происходить', ' ', 'нападение', '\n']


In [13]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
print(stop_words)

{'его', 'такой', 'этой', 'уж', 'чего', 'для', 'себе', 'лучше', 'я', 'там', 'ж', 'им', 'без', 'чтобы', 'него', 'между', 'да', 'даже', 'тебя', 'до', 'конечно', 'если', 'мы', 'ничего', 'нибудь', 'наконец', 'более', 'этом', 'или', 'совсем', 'к', 'они', 'ей', 'тут', 'этот', 'может', 'что', 'вы', 'на', 'все', 'опять', 'будет', 'всю', 'за', 'с', 'нельзя', 'вас', 'какая', 'она', 'где', 'тоже', 'то', 'куда', 'но', 'про', 'о', 'когда', 'под', 'мой', 'он', 'зачем', 'над', 'будто', 'нее', 'впрочем', 'и', 'так', 'кто', 'тем', 'два', 'чуть', 'ну', 'не', 'а', 'от', 'по', 'ли', 'как', 'быть', 'их', 'раз', 'перед', 'том', 'всегда', 'моя', 'ее', 'них', 'у', 'эту', 'здесь', 'ни', 'только', 'один', 'вдруг', 'нас', 'после', 'еще', 'того', 'себя', 'уже', 'почти', 'этого', 'какой', 'из', 'потому', 'со', 'ним', 'вам', 'надо', 'свою', 'три', 'мне', 'вот', 'теперь', 'ты', 'чтоб', 'эти', 'было', 'ней', 'был', 'ведь', 'ему', 'были', 'тогда', 'чем', 'всех', 'меня', 'больше', 'сам', 'при', 'бы', 'в', 'можно', 'друг

In [14]:
import re
def lemmatize_filter(mstem_instance, text, stop_words):
    lemmatized = pd.Series([''.join(mstem_instance.lemmatize(text_row)) for text_row in text])
    filtered = []
    for row in lemmatized:
        filtered.append(' '.join([word for word in re.split(" |, |\n |:", row) if word and word not in stop_words]))
    return pd.Series(filtered)

In [17]:
all_lemmas = lemmatize_filter(m, news_df['title'], stop_words)

In [12]:
all_lemmas

0         полиция стамбул собираться штурмовать ночной к...
1         расстрел посетитель ночной клуб стамбул попада...
2         губернатор стамбул нападение ночной клуб погиб...
3         губернатор стамбул называть террористический а...
4         очевидец ночной клуб стамбул проходить беспреп...
5         турецкий клуб который атаковать террорист серь...
6         опубликовывать видео клуб "рейн" несколько час...
7            турецкий сми запрещать писать теракт стамбул\n
8         губернатор стамбул рассказывать нападение клуб...
9         опубликовывать видео начало атака ночной клуб ...
10        власть стамбул озвучивать официальный версия н...
11            сша небо сталкиваться легкомоторный самолет\n
12        руководство клуб "рейн" предупреждать подготов...
13        экс-голкипер ска бобровский становиться герой ...
14         четыре человек пострадать взрыв петарда москва\n
15        сми среди ранить ночной клуб стамбул иностранец\n
16        обама могерини столтенберг выр

In [13]:
news_df['title'].head()

0    Полиция Стамбула собирается штурмовать ночной ...
1    Расстрел посетителей в ночном клубе в Стамбуле...
2    Губернатор Стамбула: При нападении на ночной к...
3    Губернатор Стамбула назвал террористической ат...
4    Очевидцы: В ночной клуб в Стамбуле проходили б...
Name: title, dtype: object

In [18]:
news_df['title'] = all_lemmas

In [19]:
news_df['title'].head()

0    полиция стамбул собираться штурмовать ночной к...
1    расстрел посетитель ночной клуб стамбул попада...
2    губернатор стамбул нападение ночной клуб погиб...
3    губернатор стамбул называть террористический а...
4    очевидец ночной клуб стамбул проходить беспреп...
Name: title, dtype: object

In [20]:
# json parser
def json_parse(json_object, concat_str="\n"):
    text_from_json = [e["value"]["blocks"][0]["text"] for e in json_object if e["type"] == "TEXT" or e["type"] == "SUBTITLE"]
    return concat_str.join(text_from_json)

In [21]:
# html cleaner
def html_clean(html_string):
    from bs4 import BeautifulSoup
    return BeautifulSoup(html_string, "html.parser").get_text().replace(u'\xa0', u' ')

In [22]:
import math
def extract_text(raw_text):
    counter = 0
    extracted = []
    for row in raw_text:
        if row.__class__.__name__ == 'float':
            extracted.append("")
            continue
        j = get_json(row)
        counter += 1
        if j is None: # html
            extracted.append(html_clean(row))
        else: # json
            extracted.append(json_parse(j))
        pass
    return pd.Series(extracted)

In [23]:
try_ex = extract_text(news_df['content'])

/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:282: UserWarning: "https://life.ru/oskarvote/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:282: UserWarning: "https://projects.life.ru/apocalypse/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:282: UserWarning: "https://vk.com/videos-24199209?z=video-24199209_456242732%2Fclub24199209%2Fpl_-24199209_-2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind 

In [24]:
news_df['content'] = try_ex

In [25]:
print(news_df[news_df['content'] != ""].shape)
print(news_df.shape)

(102205, 5)
(103327, 5)


In [21]:
# TODO: should work?
# news_df = news_df[news_df['content'] != ""]
# news_df = news_df[news_df['content'] != " "]

In [26]:
news_df.head()

,post_id,published_at,title,subtitle,content
0,1,2017-01-01T00:05:00Z,полиция стамбул собираться штурмовать ночной к...,"Согласно последним данным, в здании может нахо...",Полиция Стамбула собирается штурмовать ночной ...
1,2,2017-01-01T00:07:00Z,расстрел посетитель ночной клуб стамбул попада...,"Ранее сообщалось, что полицейские собираются ш...","Опубликовано видео, где злоумышленник расстрел..."
2,3,2017-01-01T00:21:00Z,губернатор стамбул нападение ночной клуб погиб...,"Ранее сообщалось, что от 40 до 60 человек полу...","Губернатор Стамбула Васип Шахин заявил, что пр..."
3,4,2017-01-01T00:29:00Z,губернатор стамбул называть террористический а...,"Согласно последним данным, погибло минимум 35 ...",Губернатор Стамбула Васип Шахин назвал террори...
4,5,2017-01-01T00:47:00Z,очевидец ночной клуб стамбул проходить беспреп...,Ранее губернатор Стамбула Васип Шахин назвал п...,"Посетители стамбульского клуба ""Рейна"", на кот..."


In [27]:
news_df.shape[0]

103327

In [24]:
news_df.to_csv('news_similarity_train_upd.csv', sep=',')

In [43]:
news_df.columns

Index(['Unnamed: 0', 'post_id', 'published_at', 'title', 'subtitle',
       'content'],
      dtype='object')

In [28]:
new_content = lemmatize_filter(m, news_df['content'], stop_words)

In [39]:
new_content

0         полиция стамбул собираться штурмовать ночной к...
1         опубликовывать видео злоумышленник расстрелива...
2         губернатор стамбул васип шахин заявлять нападе...
3         губернатор стамбул васип шахин называть террор...
4         посетитель стамбульский клуб "рейн" который со...
5         клуб "рейн" который сегодня совершать вооружат...
6         опубликовывать видео стамбульский клуб "рейн" ...
7         официальный власть турция экстренно издали нов...
8         слово губернатор стамбул васип шахин атака эли...
9         сеть появляться видео начало атака ночной клуб...
10        губернатор стамбул васип шахин рассказывать по...
11        американский город маккинни небо сталкиваться ...
12        хозяин ночной клуб "рейн" стамбул мехмет кочар...
13        бывший голкипер петербургский ска сергей бобро...
14        медучреждение подмосковье обращаться четыре че...
15        среди получать ранение клуб "рейн" стамбул гра...
16        президент сша барак обама генс

In [29]:
news_df['content'] = new_content

In [30]:
news_df.to_csv('news_similarity_train_upd.csv', sep=',')

In [31]:
sample_file = 'news_similarity_sample.csv'
sample_df = pd.read_csv(sample_file, sep=",", engine="c")

In [49]:
sample_df.columns

Index(['id', 'similarity'], dtype='object')

In [33]:
print(sample_df.head())
print(sample_df.columns)

            id  similarity
0  38277_31392         0.5
1  79621_65626         0.5
2  61134_17553         0.5
3  56944_64787         0.5
4   48365_7040         0.5
Index(['id', 'similarity'], dtype='object')


In [34]:
news_df['content'][0]

'полиция стамбул собираться штурмовать ночной клуб происходить нападение. согласно последний данные минимум злоумышленник находиться здание. ранить разный данные 40 60 человек.\nжертва становиться мало четыре человек. среди погибший — полицейский.\nво время празднество клуб "рейн" находиться 800 человек. неизвестный производить множество выстрел сообщаться взрывать граната.\n'

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
title_vect = CountVectorizer().fit(news_df['title'])
content_vect = CountVectorizer().fit(news_df['content'])
titles = title_vect.transform(news_df['title'])
contents = content_vect.transform(news_df['content'])

In [64]:
# news_df.loc[news_df['post_id'] == sample_df['id'][0].split("_")[0]]
news_df.loc[news_df['post_id'] == 38277]

,Unnamed: 0,post_id,published_at,title,subtitle,content
36512,38276,38277,2017-04-18T10:54:00Z,70% молодой водитель нравиться электромобиль\n,"Большая часть водителей, получающих права, не ...",итог опрос портал Driving-Tests большой часть ...


In [36]:
def get_index(df, post_id, col='post_id'):
    return df.index[df[col] == post_id][0]

In [37]:
# int(sample_df['id'][0].split("_")[0])

def elem(arr, index):
    return int(arr.split("_")[index])

In [61]:
get_index(news_df, 38277)

36512

In [63]:
news_df.loc[36512]

Unnamed: 0                                                  38276
post_id                                                     38277
published_at                                 2017-04-18T10:54:00Z
title              70% молодой водитель нравиться электромобиль\n
subtitle        Большая часть водителей, получающих права, не ...
content         итог опрос портал Driving-Tests большой часть ...
Name: 36512, dtype: object

In [81]:
# print(titles[36512] != title_vect.transform(news_df['title'])[36512])
# print(news_df['title'][36512])

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
# float(cosine_similarity(titles[get_index(news_df, elem(sample_df['id'][i], 0))], titles[get_index(news_df, elem(sample_df['id'][i], 1))]))

In [53]:
title_similarities = []
content_similarities = []
total_similarity = []
for id_pair in sample_df['id']:
    first_index = get_index(news_df, elem(id_pair, 0))
    second_index = get_index(news_df, elem(id_pair, 1))
    title_sim = float(cosine_similarity(titles[first_index], titles[second_index]))
    content_sim = float(cosine_similarity(contents[first_index], contents[second_index]))
    title_weight = 0.1
    if title_sim != float(0) and content_sim != float(0):
        title_sim *= title_weight
        content_sim *= float(1) - title_weight
    title_similarities.append(title_sim)
    content_similarities.append(content_sim)
    total_similarity.append(title_sim + content_sim)
    pass

In [54]:
sample_df['similarity'] = pd.Series(total_similarity)
sample_df.head()

,id,similarity
0,38277_31392,0.110062
1,79621_65626,0.360157
2,61134_17553,0.000000
3,56944_64787,0.139225
4,48365_7040,0.213156


In [55]:
sample_df.to_csv('news_ag_submission2.csv', sep=',')